<a href="https://colab.research.google.com/github/javeriiaaa/Thesis/blob/main/Bachelor_Thesis_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
pd_json=pd.read_json('Software_5.json', lines=True)

In [ ]:
pd_json

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,4,False,"10 20, 2010",A38NELQT98S4H8,0321719816,{'Format:': ' DVD-ROM'},WB Halper,I've been using Dreamweaver (and it's predeces...,A solid overview of Dreamweaver CS5,1287532800,NaN,NaN
1,4,False,"10 18, 2010",A3QJU4FEN8PQSZ,0321719816,{'Format:': ' DVD-ROM'},Grimmy,"The demo is done with the PC version, with ref...",A good value,1287360000,NaN,NaN
2,5,False,"10 16, 2010",ACJT8MUC0LRF0,0321719816,{'Format:': ' DVD-ROM'},D. Fowler,If you've been wanting to learn how to create ...,This is excellent software for those who want ...,1287187200,3,NaN
3,5,False,"10 12, 2010",AYUF7YETYOLNX,0321719816,{'Format:': ' DVD-ROM'},Bryan Newman,I've been creating websites with Dreamweaver f...,A Fantastic Overview of Dream Weaver and Web D...,1286841600,NaN,NaN
4,5,False,"10 7, 2010",A31ICLWQ9CSHRS,0321719816,{'Format:': ' DVD-ROM'},Al Swanson,I decided (after trying a number of other prod...,Excellent Tutorials!,1286409600,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
12800,4,False,"07 16, 2016",A1E50L7PCVXLN4,B01FFVDY9M,{'Platform:': ' Key Card'},Colinda,When I ordered this it was listed as Photo Edi...,File Management Software with Basic Editing Ca...,1468627200,NaN,NaN
12801,3,False,"06 17, 2017",AVU1ILDDYW301,B01HAP3NUG,NaN,G. Hearn,This software has SO much going on. Theres a ...,"Might not be for the ""novice""",1497657600,NaN,NaN
12802,4,False,"01 24, 2017",A2LW5AL0KQ9P1M,B01HAP3NUG,NaN,Dr. E,I have used both more complex and less complex...,"Great, Inexpensive Software for Those Who Have...",1485216000,NaN,NaN
12803,3,False,"06 14, 2018",AZ515FFZ7I2P7,B01HAP47PQ,{'Platform:': ' PC Disc'},Jerry Jackson Jr.,Pinnacle Studio 20 Ultimate is a perfectly ser...,Gets the job done ... but not as easy as it sh...,1528934400,NaN,NaN


In [ ]:
pd_df=pd_json

In [ ]:
pd_df.columns

Index(['overall', 'verified', 'reviewTime', 'reviewerID', 'asin', 'style',
       'reviewerName', 'reviewText', 'summary', 'unixReviewTime', 'vote',
       'image'],
      dtype='object')

In [ ]:
columns=['overall', 'verified', 'reviewTime', 'style',
       'reviewerName', 'summary', 'unixReviewTime', 'vote',
       'image']

In [ ]:
pd_rev=pd_df.drop(columns=columns, axis=1)

In [ ]:
pd_rev

,reviewerID,asin,reviewText
0,A38NELQT98S4H8,0321719816,I've been using Dreamweaver (and it's predeces...
1,A3QJU4FEN8PQSZ,0321719816,"The demo is done with the PC version, with ref..."
2,ACJT8MUC0LRF0,0321719816,If you've been wanting to learn how to create ...
3,AYUF7YETYOLNX,0321719816,I've been creating websites with Dreamweaver f...
4,A31ICLWQ9CSHRS,0321719816,I decided (after trying a number of other prod...
...,...,...,...
12800,A1E50L7PCVXLN4,B01FFVDY9M,When I ordered this it was listed as Photo Edi...
12801,AVU1ILDDYW301,B01HAP3NUG,This software has SO much going on. Theres a ...
12802,A2LW5AL0KQ9P1M,B01HAP3NUG,I have used both more complex and less complex...
12803,AZ515FFZ7I2P7,B01HAP47PQ,Pinnacle Studio 20 Ultimate is a perfectly ser...


In [ ]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 5.5 MB/s 
     |████████████████████████████████| 895 kB 53.9 MB/s 
     |████████████████████████████████| 6.8 MB 43.1 MB/s 
     |████████████████████████████████| 596 kB 54.0 MB/s 
     |████████████████████████████████| 67 kB 4.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


#**configure model** https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

In [ ]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []


    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
# model = AutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)
# tokenizer.save_pretrained(MODEL)



Downloading:   0%|          | 0.00/747 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [ ]:
def find_max(array):

  print(max(array))
  # print(max(array).index)




In [ ]:
find_max([34,56,78])

78


In [ ]:
# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)




Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

In [ ]:
def calculate_sentiment_label(text):
  text = preprocess(text)
  encoded_input = tokenizer(text, return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)

  # # TF
  # model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
  # model.save_pretrained(MODEL)

  # text = "Good night 😊"
  # encoded_input = tokenizer(text, return_tensors='tf')
  # output = model(encoded_input)
  # scores = output[0][0].numpy()
  # scores = softmax(scores)

  # index_max = np.argmax(scores)
  # value=scores[index_max]
  ranking = np.argsort(scores)
  ranking = ranking[::-1]

  # label=labels[ranking[index_max]]


  for i in range(scores.shape[0]):

      l = labels[ranking[i]]
      s = scores[ranking[i]]


      return l
def calculate_sentiment_value(text):
  text = preprocess(text)
  encoded_input = tokenizer(text, return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)

  # # TF
  # model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
  # model.save_pretrained(MODEL)

  # text = "Good night 😊"
  # encoded_input = tokenizer(text, return_tensors='tf')
  # output = model(encoded_input)
  # scores = output[0][0].numpy()
  # scores = softmax(scores)

  # index_max = np.argmax(scores)
  # value=scores[index_max]
  ranking = np.argsort(scores)
  ranking = ranking[::-1]

  # label=labels[ranking[index_max]]


  for i in range(scores.shape[0]):

      l = labels[ranking[i]]
      s = scores[ranking[i]]


      return s

In [ ]:
text='I somewhat love you'
calculate_sentiment_label(text)

'positive'

In [ ]:
array=pd.DataFrame(['I hate you', 'I love you', 'Good job', 'nah not good'], columns=['text'])

In [ ]:
# pd_rev['sentiment']=pd_rev['reviewText'].apply(lambda text:calculate_sentiment(text))

In [ ]:
# array.text.apply(lambda x: calculate_sentiment(x))

In [ ]:
#make preference array for every user. add only those items where sentiment = positive

In [ ]:
from textblob import TextBlob

In [ ]:
def sentiment_analysis(text):
  polarity=TextBlob(text).sentiment.polarity
  return polarity


In [ ]:
array['sentiment']=array.text.apply(lambda x: sentiment_analysis(x))

In [ ]:
array

,text,sentiment
0,I hate you,-0.80
1,I love you,0.50
2,Good job,0.70
3,nah not good,-0.35


In [ ]:
# pd_rev['sentiment']=pd_rev.reviewText.apply(lambda text:sentiment_analysis(text))

In [ ]:
pd_rev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12805 entries, 0 to 12804
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reviewerID  12805 non-null  object
 1   asin        12805 non-null  object
 2   reviewText  12804 non-null  object
dtypes: object(3)
memory usage: 300.2+ KB


In [ ]:
pd_rev.isnull().sum()

reviewerID    0
asin          0
reviewText    1
dtype: int64

In [ ]:
pd_rev = pd_rev.dropna(how='any',axis=0)

In [ ]:
pd_rev.isnull().sum()

reviewerID    0
asin          0
reviewText    0
dtype: int64

In [ ]:
pd_rev['sentiment']=pd_rev.reviewText.apply(lambda text:sentiment_analysis(text))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
pd_rev

,reviewerID,asin,reviewText,sentiment
0,A38NELQT98S4H8,0321719816,I've been using Dreamweaver (and it's predeces...,0.232701
1,A3QJU4FEN8PQSZ,0321719816,"The demo is done with the PC version, with ref...",0.013317
2,ACJT8MUC0LRF0,0321719816,If you've been wanting to learn how to create ...,0.193658
3,AYUF7YETYOLNX,0321719816,I've been creating websites with Dreamweaver f...,0.258266
4,A31ICLWQ9CSHRS,0321719816,I decided (after trying a number of other prod...,0.266241
...,...,...,...,...
12800,A1E50L7PCVXLN4,B01FFVDY9M,When I ordered this it was listed as Photo Edi...,0.084900
12801,AVU1ILDDYW301,B01HAP3NUG,This software has SO much going on. Theres a ...,0.289231
12802,A2LW5AL0KQ9P1M,B01HAP3NUG,I have used both more complex and less complex...,0.189735
12803,AZ515FFZ7I2P7,B01HAP47PQ,Pinnacle Studio 20 Ultimate is a perfectly ser...,-0.009615


In [ ]:
# pd_rev.rename(columns={'sentiment': 'polarity_m1'})

In [ ]:
def createLabel(value):
  if value > 0:
    return 'positive'
  else:
    return 'negative'

In [ ]:
pd_rev

,reviewerID,asin,reviewText,sentiment
0,A38NELQT98S4H8,0321719816,I've been using Dreamweaver (and it's predeces...,0.232701
1,A3QJU4FEN8PQSZ,0321719816,"The demo is done with the PC version, with ref...",0.013317
2,ACJT8MUC0LRF0,0321719816,If you've been wanting to learn how to create ...,0.193658
3,AYUF7YETYOLNX,0321719816,I've been creating websites with Dreamweaver f...,0.258266
4,A31ICLWQ9CSHRS,0321719816,I decided (after trying a number of other prod...,0.266241
...,...,...,...,...
12800,A1E50L7PCVXLN4,B01FFVDY9M,When I ordered this it was listed as Photo Edi...,0.084900
12801,AVU1ILDDYW301,B01HAP3NUG,This software has SO much going on. Theres a ...,0.289231
12802,A2LW5AL0KQ9P1M,B01HAP3NUG,I have used both more complex and less complex...,0.189735
12803,AZ515FFZ7I2P7,B01HAP47PQ,Pinnacle Studio 20 Ultimate is a perfectly ser...,-0.009615


In [ ]:
pd_rev['label_m1']=pd_rev.sentiment.apply(lambda value: createLabel(value))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
# pd_rev['label_m2']=pd_rev.reviewText.apply(lambda text: calculate_sentiment_label(text))

In [ ]:
pd_rev

,reviewerID,asin,reviewText,sentiment,label_m1
0,A38NELQT98S4H8,0321719816,I've been using Dreamweaver (and it's predeces...,0.232701,positive
1,A3QJU4FEN8PQSZ,0321719816,"The demo is done with the PC version, with ref...",0.013317,positive
2,ACJT8MUC0LRF0,0321719816,If you've been wanting to learn how to create ...,0.193658,positive
3,AYUF7YETYOLNX,0321719816,I've been creating websites with Dreamweaver f...,0.258266,positive
4,A31ICLWQ9CSHRS,0321719816,I decided (after trying a number of other prod...,0.266241,positive
...,...,...,...,...,...
12800,A1E50L7PCVXLN4,B01FFVDY9M,When I ordered this it was listed as Photo Edi...,0.084900,positive
12801,AVU1ILDDYW301,B01HAP3NUG,This software has SO much going on. Theres a ...,0.289231,positive
12802,A2LW5AL0KQ9P1M,B01HAP3NUG,I have used both more complex and less complex...,0.189735,positive
12803,AZ515FFZ7I2P7,B01HAP47PQ,Pinnacle Studio 20 Ultimate is a perfectly ser...,-0.009615,negative


In [ ]:
pd_rev_pos_only= pd_rev[pd_rev.label_m1 == 'positive']

In [ ]:
pd_rev_pos_only

,reviewerID,asin,reviewText,sentiment,label_m1
0,A38NELQT98S4H8,0321719816,I've been using Dreamweaver (and it's predeces...,0.232701,positive
1,A3QJU4FEN8PQSZ,0321719816,"The demo is done with the PC version, with ref...",0.013317,positive
2,ACJT8MUC0LRF0,0321719816,If you've been wanting to learn how to create ...,0.193658,positive
3,AYUF7YETYOLNX,0321719816,I've been creating websites with Dreamweaver f...,0.258266,positive
4,A31ICLWQ9CSHRS,0321719816,I decided (after trying a number of other prod...,0.266241,positive
...,...,...,...,...,...
12799,A5U5T6EWH90O0,B01FFVDY9M,"I am a total amateur when it comes to editing,...",0.227632,positive
12800,A1E50L7PCVXLN4,B01FFVDY9M,When I ordered this it was listed as Photo Edi...,0.084900,positive
12801,AVU1ILDDYW301,B01HAP3NUG,This software has SO much going on. Theres a ...,0.289231,positive
12802,A2LW5AL0KQ9P1M,B01HAP3NUG,I have used both more complex and less complex...,0.189735,positive


In [ ]:
user_df=pd.DataFrame(pd_rev_pos_only.reviewerID.unique(), columns={'user_id'})

In [ ]:
user_df

,user_id
0,A38NELQT98S4H8
1,A3QJU4FEN8PQSZ
2,ACJT8MUC0LRF0
3,AYUF7YETYOLNX
4,A31ICLWQ9CSHRS
...,...
1774,A1SOH83X2VW469
1775,A1H378KKY8V01U
1776,A3OAA19C65C5FT
1777,A34AN3NF8P7PJ


In [ ]:
user_df['preferences']=pd_rev_pos_only.groupby('reviewerID')['asin'].apply(list)

In [ ]:
# grouped_df = df.groupby("Column1")

# grouped_lists = grouped_df["Column2"].apply(list)

# grouped_lists = grouped_lists.reset_index()


In [ ]:
# grouped_df=pd_rev_pos_only.groupby('reviewerID')['asin'].apply(list)

In [ ]:
grouped_df = pd_rev_pos_only.groupby("reviewerID")

grouped_lists = grouped_df["asin"].apply(list)

grouped_lists = grouped_lists.reset_index()

In [ ]:
print(grouped_lists)

          reviewerID                                               asin
0     A100UD67AHFODS   [B005S4Y65I, B009CCVMO0, B00BR082FW, B00F8K9KZS]
1     A105S56ODHGJEK  [B000UJUJ7U, B000WR2F2M, B000X86ZAS, B0013O54O...
2     A1075X1Q4M3S78  [B005FIWT74, B008RA5A00, B00E6LJ2SA, B01326J62...
3     A10C5CJK1YKGV0  [B0013OAHTG, B0013OAHTG, B001AFCWBO, B001AFFZM...
4     A10CRW7XRJBJ2G               [B00FFINOWS, B00PG8FFFQ, B01637RMYU]
...              ...                                                ...
1774   AZD8SMNGQI98O  [B002JB3BC2, B002PDPIJI, B002X8V326, B004YTG20...
1775   AZQGJ5CEAJGXB  [B008U5EL5C, B009ENDZ4C, B00N4OLGP2, B01CS8RSI...
1776   AZW10G02DNJI4  [B00EZPXYP4, B00M9GTL6M, B00MV9ELBM, B00MYXTCN...
1777   AZYJE40XW6MFG  [0982697813, B004IKEKWC, B0056CZATI, B0056CZC2...
1778   AZZ5ASC403N74   [0763855553, B008414ZYY, B009SPL2K0, B00F9ZQQ8Q]

[1779 rows x 2 columns]


In [ ]:
user_df=pd.DataFrame({'preferences' : pd_rev_pos_only.groupby("reviewerID")["asin"].apply(list)}).reset_index()

In [ ]:
user_df

,reviewerID,preferences
0,A100UD67AHFODS,"[B005S4Y65I, B009CCVMO0, B00BR082FW, B00F8K9KZS]"
1,A105S56ODHGJEK,"[B000UJUJ7U, B000WR2F2M, B000X86ZAS, B0013O54O..."
2,A1075X1Q4M3S78,"[B005FIWT74, B008RA5A00, B00E6LJ2SA, B01326J62..."
3,A10C5CJK1YKGV0,"[B0013OAHTG, B0013OAHTG, B001AFCWBO, B001AFFZM..."
4,A10CRW7XRJBJ2G,"[B00FFINOWS, B00PG8FFFQ, B01637RMYU]"
...,...,...
1774,AZD8SMNGQI98O,"[B002JB3BC2, B002PDPIJI, B002X8V326, B004YTG20..."
1775,AZQGJ5CEAJGXB,"[B008U5EL5C, B009ENDZ4C, B00N4OLGP2, B01CS8RSI..."
1776,AZW10G02DNJI4,"[B00EZPXYP4, B00M9GTL6M, B00MV9ELBM, B00MYXTCN..."
1777,AZYJE40XW6MFG,"[0982697813, B004IKEKWC, B0056CZATI, B0056CZC2..."


In [ ]:
pd_df

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,4,False,"10 20, 2010",A38NELQT98S4H8,0321719816,{'Format:': ' DVD-ROM'},WB Halper,I've been using Dreamweaver (and it's predeces...,A solid overview of Dreamweaver CS5,1287532800,NaN,NaN
1,4,False,"10 18, 2010",A3QJU4FEN8PQSZ,0321719816,{'Format:': ' DVD-ROM'},Grimmy,"The demo is done with the PC version, with ref...",A good value,1287360000,NaN,NaN
2,5,False,"10 16, 2010",ACJT8MUC0LRF0,0321719816,{'Format:': ' DVD-ROM'},D. Fowler,If you've been wanting to learn how to create ...,This is excellent software for those who want ...,1287187200,3,NaN
3,5,False,"10 12, 2010",AYUF7YETYOLNX,0321719816,{'Format:': ' DVD-ROM'},Bryan Newman,I've been creating websites with Dreamweaver f...,A Fantastic Overview of Dream Weaver and Web D...,1286841600,NaN,NaN
4,5,False,"10 7, 2010",A31ICLWQ9CSHRS,0321719816,{'Format:': ' DVD-ROM'},Al Swanson,I decided (after trying a number of other prod...,Excellent Tutorials!,1286409600,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
12800,4,False,"07 16, 2016",A1E50L7PCVXLN4,B01FFVDY9M,{'Platform:': ' Key Card'},Colinda,When I ordered this it was listed as Photo Edi...,File Management Software with Basic Editing Ca...,1468627200,NaN,NaN
12801,3,False,"06 17, 2017",AVU1ILDDYW301,B01HAP3NUG,NaN,G. Hearn,This software has SO much going on. Theres a ...,"Might not be for the ""novice""",1497657600,NaN,NaN
12802,4,False,"01 24, 2017",A2LW5AL0KQ9P1M,B01HAP3NUG,NaN,Dr. E,I have used both more complex and less complex...,"Great, Inexpensive Software for Those Who Have...",1485216000,NaN,NaN
12803,3,False,"06 14, 2018",AZ515FFZ7I2P7,B01HAP47PQ,{'Platform:': ' PC Disc'},Jerry Jackson Jr.,Pinnacle Studio 20 Ultimate is a perfectly ser...,Gets the job done ... but not as easy as it sh...,1528934400,NaN,NaN


In [ ]:
df_summary=pd.DataFrame(pd_df[['asin', 'summary']])

In [ ]:
df_summary

,asin,summary
0,0321719816,A solid overview of Dreamweaver CS5
1,0321719816,A good value
2,0321719816,This is excellent software for those who want ...
3,0321719816,A Fantastic Overview of Dream Weaver and Web D...
4,0321719816,Excellent Tutorials!
...,...,...
12800,B01FFVDY9M,File Management Software with Basic Editing Ca...
12801,B01HAP3NUG,"Might not be for the ""novice"""
12802,B01HAP3NUG,"Great, Inexpensive Software for Those Who Have..."
12803,B01HAP47PQ,Gets the job done ... but not as easy as it sh...


In [ ]:
df_summary.dropna()

,asin,summary
0,0321719816,A solid overview of Dreamweaver CS5
1,0321719816,A good value
2,0321719816,This is excellent software for those who want ...
3,0321719816,A Fantastic Overview of Dream Weaver and Web D...
4,0321719816,Excellent Tutorials!
...,...,...
12800,B01FFVDY9M,File Management Software with Basic Editing Ca...
12801,B01HAP3NUG,"Might not be for the ""novice"""
12802,B01HAP3NUG,"Great, Inexpensive Software for Those Who Have..."
12803,B01HAP47PQ,Gets the job done ... but not as easy as it sh...


In [ ]:
grouped_df2 = df_summary.groupby("asin")

grouped_lists2 = grouped_df2["summary"].apply(list)

grouped_lists2 = grouped_lists2.reset_index()

In [ ]:
grouped_df2

In [ ]:
print(grouped_lists2)

           asin                                            summary
0    0321700945  [Learn Adobe Photoshop Lightroom 3 by Video (L...
1    0321719816  [A solid overview of Dreamweaver CS5, A good v...
2    0321719824  [A helpful introduction to Flash CS5 - a bit d...
3    0763855553  [A disappointment, Still the Standard, and eas...
4    0982697813  [Glo is good to go with its improvements to th...
..          ...                                                ...
797  B01F7RJHIQ  [No real difference to 2007, Lipstick on a Pig...
798  B01FFVDY9M  [Powerful Software For Its Price Point, Excell...
799  B01H39M7ME  [One Star, ??????, One Star, Didn't work, Five...
800  B01HAP3NUG  [Not user friendly, and lacks some key feature...
801  B01HAP47PQ  [Works well for novice users and higher end al...

[802 rows x 2 columns]


In [ ]:
df_summary=df_summary=pd.DataFrame({'summary' : df_summary.groupby("asin")["summary"].apply(list)}).reset_index()

In [ ]:
df_summary

,asin,summary
0,0321700945,[Learn Adobe Photoshop Lightroom 3 by Video (L...
1,0321719816,"[A solid overview of Dreamweaver CS5, A good v..."
2,0321719824,[A helpful introduction to Flash CS5 - a bit d...
3,0763855553,"[A disappointment, Still the Standard, and eas..."
4,0982697813,[Glo is good to go with its improvements to th...
...,...,...
797,B01F7RJHIQ,"[No real difference to 2007, Lipstick on a Pig..."
798,B01FFVDY9M,"[Powerful Software For Its Price Point, Excell..."
799,B01H39M7ME,"[One Star, ??????, One Star, Didn't work, Five..."
800,B01HAP3NUG,"[Not user friendly, and lacks some key feature..."


In [ ]:
df_summary.dtypes

asin       object
summary    object
dtype: object

In [ ]:
df_summary["summary"] = df_summary["summary"].astype(str)

In [ ]:
df_summary

,asin,summary
0,0321700945,['Learn Adobe Photoshop Lightroom 3 by Video (...
1,0321719816,"['A solid overview of Dreamweaver CS5', 'A goo..."
2,0321719824,['A helpful introduction to Flash CS5 - a bit ...
3,0763855553,"['A disappointment', 'Still the Standard', 'an..."
4,0982697813,['Glo is good to go with its improvements to t...
...,...,...
797,B01F7RJHIQ,"['No real difference to 2007', 'Lipstick on a ..."
798,B01FFVDY9M,"['Powerful Software For Its Price Point', 'Exc..."
799,B01H39M7ME,"['One Star', '??????', 'One Star', ""Didn't wor..."
800,B01HAP3NUG,"['Not user friendly, and lacks some key featur..."


In [ ]:
#clean the summary text

In [ ]:
import string

In [ ]:
def clean_textR1(texts):
  for text in texts:
    text=text.lower()
    text=re.sub('@\w*', '', text)
    text=re.sub('\[.*?\]', '',text)
    text=re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text=re.sub('\w*\d\w*', '', text)
  return texts

round1=lambda x:clean_textR1(x)

In [ ]:
def clean_textR2(texts):
  for text in texts:
    text=re.sub('[''""_]', '',text)
    text=re.sub('\n', '',text)

  return texts
round2=lambda x:clean_textR2(x)

In [ ]:
!pip install nltk

In [ ]:

import nltk

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import re
from nltk.tokenize import RegexpTokenizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
stop_words=stopwords.words('english')
stemmer=SnowballStemmer('english')
tokenizer=RegexpTokenizer(r'\w+')

In [ ]:
df_sum_c=df_summary.copy()

In [ ]:
df_sum_c

,asin,summary
0,0321700945,['Learn Adobe Photoshop Lightroom 3 by Video (...
1,0321719816,"['A solid overview of Dreamweaver CS5', 'A goo..."
2,0321719824,['A helpful introduction to Flash CS5 - a bit ...
3,0763855553,"['A disappointment', 'Still the Standard', 'an..."
4,0982697813,['Glo is good to go with its improvements to t...
...,...,...
797,B01F7RJHIQ,"['No real difference to 2007', 'Lipstick on a ..."
798,B01FFVDY9M,"['Powerful Software For Its Price Point', 'Exc..."
799,B01H39M7ME,"['One Star', '??????', 'One Star', ""Didn't wor..."
800,B01HAP3NUG,"['Not user friendly, and lacks some key featur..."


In [ ]:
df_sum_c.summary=df_sum_c.summary.apply(lambda x: clean_textR1(x))

In [ ]:
df_sum_c.summary=df_sum_c.summary.apply(lambda x: clean_textR2(x))

In [ ]:
df_sum_c.loc[df_sum_c['asin'] == '0321700945']

,asin,summary
0,0321700945,['Learn Adobe Photoshop Lightroom 3 by Video (...


In [ ]:
df_sum_c['summary']


0      ['Learn Adobe Photoshop Lightroom 3 by Video (...
1      ['A solid overview of Dreamweaver CS5', 'A goo...
2      ['A helpful introduction to Flash CS5 - a bit ...
3      ['A disappointment', 'Still the Standard', 'an...
4      ['Glo is good to go with its improvements to t...
                             ...                        
797    ['No real difference to 2007', 'Lipstick on a ...
798    ['Powerful Software For Its Price Point', 'Exc...
799    ['One Star', '??????', 'One Star', "Didn't wor...
800    ['Not user friendly, and lacks some key featur...
801    ['Works well for novice users and higher end a...
Name: summary, Length: 802, dtype: object

In [ ]:

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
lem=nltk.WordNetLemmatizer()

In [ ]:
# def generateLemTokens(sentences):
#   tokens=[]
#   sentences=[re.sub('@\w*', '', sentences)]
#   for sent in sentences:
#     for word in sent:
#       if word not in stop_words:
#         tokens.append(word)
#   return tokenizer.tokenize("".join(tokens))\
def tokenize(sentences):
  tokens=[]
  for word in sentences:
      if word not in stop_words:
        tokens.append(word)
  return tokenizer.tokenize("".join(tokens))

In [ ]:
df_sum_c.summary=df_sum_c.summary.apply(lambda x: tokenize(x))

In [ ]:
df_sum_c

,asin,summary
0,0321700945,"[Lern, Abe, Phhp, Lghr, 3, b, Ve, Lern, b, Ve,..."
1,0321719816,"[A, l, vervew, f, Drewever, CS5, A, g, vlue, T..."
2,0321719824,"[A, helpful, nrucn, Flh, CS5, b, r, perhp, bu,..."
3,0763855553,"[A, ppnen, Sll, he, Snr, n, el, crrupe, Oulk, ..."
4,0982697813,"[Gl, g, g, wh, prveen, he, fwre, u, cn, Expere..."
...,...,...
797,B01F7RJHIQ,"[N, rel, fference, 2007, Lpck, n, Pg, MS, Offc..."
798,B01FFVDY9M,"[Pwerful, Sfwre, Fr, I, Prce, Pn, Excellen, Ph..."
799,B01H39M7ME,"[One, Sr, One, Sr, Dn, wrk, Fve, Sr, Fve, Sr, ..."
800,B01HAP3NUG,"[N, uer, frenl, n, lck, e, ke, feure, u, hul, ..."


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf=TfidfVectorizer()

In [ ]:
df_sum_c['summary']=df_sum_c['summary'].apply(lambda x : ''.join(x))

In [ ]:
df_sum_c

,asin,summary
0,0321700945,LernAbePhhpLghr3bVeLernbVeAbluelhebeIfnelfgngb...
1,0321719816,AlvervewfDreweverCS5AgvlueThexcellenfwrefrhewh...
2,0321719824,AhelpfulnrucnFlhCS5brperhpbuverclernuefulexpln...
3,0763855553,AppnenSllheSnrnelcrrupeOulklleeupBevlueepecpen...
4,0982697813,GlggwhprveenhefwreucnExperenceheBbleLkeNeverBe...
...,...,...
797,B01F7RJHIQ,Nrelfference2007LpcknPgMSOffceSuenwulncveFveSr...
798,B01FFVDY9M,PwerfulSfwreFrIPrcePnExcellenPhEngMngeenPrgrAf...
799,B01H39M7ME,OneSrOneSrDnwrkFveSrFveSrThreeSr
800,B01HAP3NUG,NuerfrenlnlckekefeureuhulbewrefVernceveengfwre...


In [ ]:
tfidf_matrix = tfidf.fit_transform(df_sum_c['summary'])

In [ ]:
tfidf_matrix.shape

(802, 802)

In [ ]:
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
cosine_sim[0]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [ ]:
df_sum_c = df_sum_c.reset_index()
asins = df_sum_c['asin']
indices = pd.Series(df_sum_c.index, index=df_sum_c['asin'])

In [ ]:
def get_recommendations(asin):
    idx = indices[asin]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:5]
    movie_indices = [i[0] for i in sim_scores]
    return asins.iloc[movie_indices]

In [ ]:
get_recommendations('0321700945').head(10)

1    0321719816
2    0321719824
3    0763855553
4    0982697813
Name: asin, dtype: object

In [ ]:
user_df

,reviewerID,preferences
0,A100UD67AHFODS,"[B005S4Y65I, B009CCVMO0, B00BR082FW, B00F8K9KZS]"
1,A105S56ODHGJEK,"[B000UJUJ7U, B000WR2F2M, B000X86ZAS, B0013O54O..."
2,A1075X1Q4M3S78,"[B005FIWT74, B008RA5A00, B00E6LJ2SA, B01326J62..."
3,A10C5CJK1YKGV0,"[B0013OAHTG, B0013OAHTG, B001AFCWBO, B001AFFZM..."
4,A10CRW7XRJBJ2G,"[B00FFINOWS, B00PG8FFFQ, B01637RMYU]"
...,...,...
1774,AZD8SMNGQI98O,"[B002JB3BC2, B002PDPIJI, B002X8V326, B004YTG20..."
1775,AZQGJ5CEAJGXB,"[B008U5EL5C, B009ENDZ4C, B00N4OLGP2, B01CS8RSI..."
1776,AZW10G02DNJI4,"[B00EZPXYP4, B00M9GTL6M, B00MV9ELBM, B00MYXTCN..."
1777,AZYJE40XW6MFG,"[0982697813, B004IKEKWC, B0056CZATI, B0056CZC2..."


In [ ]:
user_df['similar_items']=user_df['preferences'].apply(lambda x: list(map(get_recommendations, x)))

In [ ]:
user_df.iloc[325]['preferences']

['0982697813',
 'B0046ED0N0',
 'B007QYZ08A',
 'B00EZPXVWU',
 'B00JC5Y6WM',
 'B00MV9ELBM',
 'B00MYXTCNC',
 'B00N4OLGP2']

In [ ]:
user_df.iloc[325]['similar_items']

[0    0321700945
 1    0321719816
 2    0321719824
 3    0763855553
 Name: asin, dtype: object, 0    0321700945
 1    0321719816
 2    0321719824
 3    0763855553
 Name: asin, dtype: object, 0    0321700945
 1    0321719816
 2    0321719824
 3    0763855553
 Name: asin, dtype: object, 0    0321700945
 1    0321719816
 2    0321719824
 3    0763855553
 Name: asin, dtype: object, 0    0321700945
 1    0321719816
 2    0321719824
 3    0763855553
 Name: asin, dtype: object, 0    0321700945
 1    0321719816
 2    0321719824
 3    0763855553
 Name: asin, dtype: object, 0    0321700945
 1    0321719816
 2    0321719824
 3    0763855553
 Name: asin, dtype: object, 0    0321700945
 1    0321719816
 2    0321719824
 3    0763855553
 Name: asin, dtype: object]

In [ ]:
user_df

,reviewerID,preferences,similar_items
0,A100UD67AHFODS,"[B005S4Y65I, B009CCVMO0, B00BR082FW, B00F8K9KZS]","[[0321700945, 0321719816, 0321719824, 07638555..."
1,A105S56ODHGJEK,"[B000UJUJ7U, B000WR2F2M, B000X86ZAS, B0013O54O...","[[0321700945, 0321719816, 0321719824, 07638555..."
2,A1075X1Q4M3S78,"[B005FIWT74, B008RA5A00, B00E6LJ2SA, B01326J62...","[[0321700945, 0321719816, 0321719824, 07638555..."
3,A10C5CJK1YKGV0,"[B0013OAHTG, B0013OAHTG, B001AFCWBO, B001AFFZM...","[[0321700945, 0321719816, 0321719824, 07638555..."
4,A10CRW7XRJBJ2G,"[B00FFINOWS, B00PG8FFFQ, B01637RMYU]","[[0321700945, 0321719816, 0321719824, 07638555..."
...,...,...,...
1774,AZD8SMNGQI98O,"[B002JB3BC2, B002PDPIJI, B002X8V326, B004YTG20...","[[0321700945, 0321719816, 0321719824, 07638555..."
1775,AZQGJ5CEAJGXB,"[B008U5EL5C, B009ENDZ4C, B00N4OLGP2, B01CS8RSI...","[[0321700945, 0321719816, 0321719824, 07638555..."
1776,AZW10G02DNJI4,"[B00EZPXYP4, B00M9GTL6M, B00MV9ELBM, B00MYXTCN...","[[0321700945, 0321719816, 0321719824, 07638555..."
1777,AZYJE40XW6MFG,"[0982697813, B004IKEKWC, B0056CZATI, B0056CZC2...","[[0321700945, 0321719816, 0321719824, 07638555..."


# ***Validate Recommendations***

In [ ]:
user_df.iloc[10]['preferences']

['B002DHGMK0',
 'B0079KJB54',
 'B008S0IP38',
 'B00F9ZQQ8Q',
 'B00IIL0SCS',
 'B00L13X6QA',
 'B00MCLGAAO',
 'B00UB76290']

In [ ]:
user_df.iloc[10]['similar_items'][4]

0    0321700945
1    0321719816
2    0321719824
3    0763855553
Name: asin, dtype: object

Corel VideoStudio is a video editing software package for Microsoft Windows

In [ ]:
pd_rev_pos_only.loc[pd_rev_pos_only['asin'] == 'B00IIL0SCS'].head(20).reviewText

9011    I've been using Video Studio Pro for the last ...
9013    I just find this software big and bloated for ...
9014    I had a good first impression with the program...
9015    I've now used VideoStudio x4, x5, & x6 and lik...
9016    All the bells and whistles of the big boys bra...
9017    I'm still learning how to use this so I'm not ...
9018    I've always liked Corel software, and this was...
9019    I don't do a lot of fancy stuff with videos, s...
9020    Ive used Roxios products for years to make hom...
9022    Corel has really done it this year.  We see th...
9023    I use several different tools for editing vide...
9024    Corel VideoStudio is a great, affordable and u...
9025    As a hobby, my buddies and I started a youtube...
9026    After using Corel Ulead Video Studio 11.5 for ...
9027    My computer exceeds the minimum requirements, ...
9028    <div id="video-block-R260PRR3RI016H" class="a-...
9029    I have several Corel products including this o...
Name: reviewTe

# Recommendations

Lightroom is the cloud-based service that gives you everything you need to create, edit, organize, store, and share your photos across any device

In [ ]:
pd_rev_pos_only.loc[pd_rev_pos_only['asin'] == '0321700945'].head(10).reviewText

11       As someone who has just upgraded from Lightroo...
12       Adobe approved "Adobe Photoshop Lightroom 3, L...
13       This is a greate collection of videos on the d...
14       There are over 100 video lessons here. Most us...
15       I am not an avid Lightroom user, although I ha...
16       I am a long time user of Photoshop Lightroom, ...
12192    Sometimes pouring over page after page is diff...
12193    I am a visual learner.  I can read a book many...
Name: reviewText, dtype: object

5 Software for Windows is a powerful, wide-ranging software application that's used to create immersive, interactive websites and also applications for websites and mobile devices. It delivers video (via the included Adobe Media Encoder) to the Web, tablets, TV, and smart phones

In [ ]:
pd_rev_pos_only.loc[pd_rev_pos_only['asin'] == '0321719824'].head(10).reviewText

17       The "Learn by Video" program for Flash CS5 wou...
18       The presenter here sounds much more natural th...
19       This certified associate courseware comes as a...
20       This was the first Learn by Video series cours...
12194    I've tried to learn Flash using books several ...
12195    I've been using Adobe Creative Suite applicati...
12196    I've been teaching media at the college level ...
12197    If you learn best by doing rather than just re...
12198    I found this instruction thorough and actually...
Name: reviewText, dtype: object

In [ ]:
pd_rev_pos_only.loc[pd_rev_pos_only['asin'] == '0763855553'].head(20).reviewText

22    Microsoft Office is still the standard all the...
23    Been using Office for over twenty years.  Stil...
24    Always liked Office Products.  Expensive for a...
25    I have been using MS Office professional for d...
26    This is really a great buy.  It breaks down to...
27    The software is well known and excellent.  It ...
28    The bottom line:  I like this deal much better...
29    I have this running on my Macbook Air and two ...
30    not that easy to understand or install, docume...
32    We got this for 2 Macs and 2 Windows machines....
33                      What can you say great product.
34                               Has been very helpful.
36    As soon as the iPad Office 365 apps got releas...
37    I Bought This So I Could Get Full Use Of The O...
38    The only real question is - does this card wor...
40    Office 365 is less than $100 per year for 5 us...
41    I really thought this was one of those program...
42    I guess it all depends on your needs. The 

Pinnacle Systems, Inc. is a California-based American manufacturer of digital video hardware and software for the mainstream and broadcast markets.

In [ ]:
pd_rev_pos_only.loc[pd_rev_pos_only['asin'] == 'B01HAP3NUG'].head(10).reviewText

12167    I've noticed some of the reviews here seem to ...
12168    Pinnacle Studio 20 Plus is a real nice video e...
12169    I'm not the most advanced video editor. I take...
12170    I've been trying to use this software for over...
12171    Get your geeky friend if you're a novice (like...
12172    Easy to install and for me fairly user friendl...
12173    I was able to pick up this application quickly...
12174    If you're looking for solid video editing at a...
12175    I'm a long time, though novice user, of Pinnac...
12176    Pinnacle Studio 20 Plus is a good entry in the...
Name: reviewText, dtype: object


<h1>Office 2016.</h1>

In [ ]:
pd_rev_pos_only.loc[pd_rev_pos_only['asin'] == 'B000X86ZAS'].head(10).reviewText

2149    Overall this office suite is excellent.  A wor...
2150    This update has been a long time coming, but w...
2151    When it comes to office applications including...
2152    Microsoft Office 2008 for Mac provides a fully...
2153    I've owned Mac's since the original 128K Mac a...
2154    Please note that this review refers to the 200...
2156    It's not a one-star product.  It can't be.  Th...
2157    Finally a Intel-native version of MS Office fo...
2158    Typical Microsoft; first thing I had to do was...
2159    Word and Excel 2008 work reasonably well. Powe...
Name: reviewText, dtype: object

<h1>Office 2008






In [ ]:
print(pd_rev_pos_only.loc[pd_rev_pos_only['asin'] == 'B0148BYIPY'].head(10).reviewText)

11291    Installed easily (through internet download wh...
11292    Somehow, some way, Microsoft has managed to sl...
11293    I have been waiting a long time for an update ...
11294    A year ago, Amazon offered me a copy of Office...
11295    If you're engaged in just about any sort of bu...
11296    Works fine, but if your using 2011, I don't th...
11297    I honestly have to say that when it does come ...
11298    Office 2016 includes Word, Excel, PowerPoint, ...
11299    It's been quite a while since the last Microso...
11300    Sorry for the late review. I wanted to fully a...
Name: reviewText, dtype: object
